In [1]:
from pathlib import Path
import os
import pandas as pd
import sys
import requests
from functools import partial
from tqdm import tqdm

import torch
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch 
from einops import rearrange, repeat
from loguru import logger
from torchmetrics import Metric, BootStrapper
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import seaborn as sns
from matplotlib.pyplot import figure
from cka import gram, centering_mat, centered_gram, unbiased_hsic_xy, MinibatchCKA, HookedCache, make_pairwise_metrics, update_metrics, get_simmat_from_metrics
import numpy as np
import re

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import timm
from timm.utils import accuracy
from torch.utils.tensorboard import SummaryWriter

from get_models import get_model

writer = SummaryWriter()

DEVICE = torch.device('cpu')
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')

### Create ImageNet validation dataset

In [2]:
# using the validation transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])

## Randomly subsample features too

data_path='/datasets01/imagenet_full_size/061417/'
dataset = torchvision.datasets.ImageFolder(data_path + 'val', transform=transform)

In [3]:
data_loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=128,
                                        shuffle=False,
                                        pin_memory=False)

log_every = 10
do_log = True

### Load pre-trained DINO, Moco-v3, MAE with linear classifiers

In [4]:
vitb16_dino = get_model('DINO', 'linear')
vitb16_mae = get_model('MAE', 'linear')
vitb16_mocov3 = get_model('MoCo-V3', 'linear')
vitb16_dino_ft = get_model('DINO', 'finetuned')
vitb16_mae_ft = get_model('MAE', 'finetuned')
vitb16_mocov3_ft = get_model('MoCo-V3', 'finetuned')
vitb16_supervised = get_model('Supervised')

Using cache found in /private/home/sshkhr/.cache/torch/hub/facebookresearch_dino_main
/private/home/sshkhr/.conda/envs/pytorch_env/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Using cache found in /private/home/sshkhr/.cache/torch/hub/facebookresearch_dino_main


In [5]:
vitb16_dino.to(DEVICE)
vitb16_mae.to(DEVICE)
vitb16_mocov3.to(DEVICE)
vitb16_dino_ft.to(DEVICE)
vitb16_mae_ft.to(DEVICE)
vitb16_mocov3_ft.to(DEVICE)
vitb16_supervised.to(DEVICE)

vitb16_supervised.eval()
vitb16_dino.eval()
vitb16_mae.eval()
vitb16_mocov3.eval()
vitb16_dino_ft.eval()
vitb16_mae_ft.eval()
vitb16_mocov3_ft.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,),

### Validate on ImageNet test set

In [6]:
def validate(dataloader, model, device):

    results = torch.zeros(len(dataloader.dataset), 2)
    rankings = torch.zeros(len(dataloader.dataset), 1000)

    with torch.no_grad():
        
        # use with one image per batch
        for it, [batch, target] in enumerate(tqdm(dataloader)):
            batch = batch.to(device)
            target = target.to(device)
            output = model(batch)
            
            batch_size = target.size(0)

            
            _, pred = output.topk(5, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
            
            results[it*batch_size:(it+1)*batch_size, 0] = correct[:1].view(-1).float() 
            results[it*batch_size:(it+1)*batch_size, 1] = correct[:5].sum(0).view(-1).float()
            

            _, rankings[it*batch_size:(it+1)*batch_size, :] = output.topk(1000, 1, True, True)

    return results, rankings

In [ ]:
moco_results, moco_ranks = validate(data_loader, vitb16_mocov3, DEVICE)

In [ ]:
dino_results, dino_ranks = validate(data_loader, vitb16_dino, DEVICE)

In [ ]:
mae_results, mae_ranks = validate(data_loader, vitb16_mae, DEVICE)

In [ ]:
moco_results_ft, moco_ranks_ft = validate(data_loader, vitb16_mocov3_ft, DEVICE)

In [ ]:
dino_results_ft, dino_ranks_ft = validate(data_loader, vitb16_dino_ft, DEVICE)

In [ ]:
mae_results_ft, mae_ranks_ft = validate(data_loader, vitb16_mae_ft, DEVICE)

In [ ]:
supervised_results, supervised_ranks = validate(data_loader, vitb16_supervised, DEVICE)

In [11]:
dic = {'supervised': [supervised_results, supervised_ranks],
       'MAE': [mae_results, mae_ranks],
       'DINO': [dino_results, dino_ranks],
       'MoCo-V3': [moco_results, moco_ranks],
       'MAE-FT': [mae_results_ft, mae_ranks_ft],
       'DINO-FT': [dino_results_ft, dino_ranks_ft],
       'MoCo-V3-FT': [moco_results_ft, moco_ranks_ft],
        }

import pickle

pickle.dump(dic, open("ranks_and_results.pkl", "wb" ))

In [6]:
import pickle

with open('ranks_and_results.pkl', 'rb') as f:
    dic = pickle.load(f)

In [7]:
supervised_results, supervised_ranks = dic['supervised']
mae_results, mae_ranks = dic['MAE']
dino_results, dino_ranks = dic['DINO']
moco_results, moco_ranks = dic['MoCo-V3']
mae_results_ft, mae_ranks_ft = dic['MAE-FT']
dino_results_ft, dino_ranks_ft = dic['DINO-FT']
moco_results_ft, moco_ranks_ft = dic['MoCo-V3-FT']


## Compare results

In [10]:
print(moco_results[:, 0].sum()/50000., moco_results[:, 1].sum()/50000.)
print(dino_results[:, 0].sum()/50000., dino_results[:, 1].sum()/50000.) # looks like DINO linear model didn't load correctly
print(mae_results[:, 0].sum()/50000., mae_results[:, 1].sum()/50000.)
print(supervised_results[:, 0].sum()/50000., supervised_results[:, 1].sum()/50000.)

print(moco_results_ft[:, 0].sum()/50000., moco_results_ft[:, 1].sum()/50000.)
print(dino_results_ft[:, 0].sum()/50000., dino_results_ft[:, 1].sum()/50000.)
print(mae_results_ft[:, 0].sum()/50000., mae_results_ft[:, 1].sum()/50000.)

tensor(0.7654) tensor(0.9312)
tensor(0.7787) tensor(0.9371)
tensor(0.6731) tensor(0.8734)
tensor(0.8094) tensor(0.9516)
tensor(0.8276) tensor(0.9607)
tensor(0.8247) tensor(0.9591)
tensor(0.8344) tensor(0.9644)


In [11]:
def get_matrix(results1, results2, top=1):
    results1 = results1[:, 0] if top==1 else results1[:,1]
    results2 = results2[:, 0] if top==1 else results2[:,1]

    matrix = np.zeros((2, 2))

    results1 = results1.numpy()
    results2 = results2.numpy()

    matrix[0, 0] = np.sum(np.logical_and(results1 == 0, results2 == 0))
    matrix[0, 1] = np.sum(np.logical_and(results1 == 0, results2 == 1))
    matrix[1, 0] = np.sum(np.logical_and(results1 == 1, results2 == 0))
    matrix[1, 1] = np.sum(np.logical_and(results1 == 1, results2 == 1))
    
    return matrix/50000.

In [8]:
from itertools import product

In [9]:
keys = list(dic.keys())

In [10]:
for index, key1 in enumerate(keys):
    for key2 in keys[index+1:]:
        matrix_1 = get_matrix(dic[key1][0], dic[key2][0], 1)
        print('Top 1', key1,'-',key2,'\n',matrix_1)
        matrix_5 = get_matrix(dic[key1][0], dic[key2][0], 5)
        print('Top 5', key1,'-',key2,'\n',matrix_5)


Top 1 supervised - MAE 
 [[0.15968 0.03096]
 [0.16726 0.6421 ]]
Top 5 supervised - MAE 
 [[0.0378  0.01056]
 [0.08882 0.86282]]
Top 1 supervised - DINO 
 [[0.15174 0.0389 ]
 [0.06952 0.73984]]
Top 5 supervised - DINO 
 [[0.03442 0.01394]
 [0.0285  0.92314]]
Top 1 supervised - MoCo-V3 
 [[0.15238 0.03826]
 [0.08222 0.72714]]
Top 5 supervised - MoCo-V3 
 [[0.03562 0.01274]
 [0.0332  0.91844]]
Top 1 supervised - MAE-FT 
 [[0.13486 0.05578]
 [0.03074 0.77862]]
Top 5 supervised - MAE-FT 
 [[0.02706 0.0213 ]
 [0.00852 0.94312]]
Top 1 supervised - DINO-FT 
 [[0.14246 0.04818]
 [0.03288 0.77648]]
Top 5 supervised - DINO-FT 
 [[0.03054 0.01782]
 [0.01034 0.9413 ]]
Top 1 supervised - MoCo-V3-FT 
 [[0.14116 0.04948]
 [0.03128 0.77808]]
Top 5 supervised - MoCo-V3-FT 
 [[0.02938 0.01898]
 [0.00988 0.94176]]
Top 1 MAE - DINO 
 [[0.18406 0.14288]
 [0.0372  0.63586]]
Top 5 MAE - DINO 
 [[0.0507  0.07592]
 [0.01222 0.86116]]
Top 1 MAE - MoCo-V3 
 [[0.1944  0.13254]
 [0.0402  0.63286]]
Top 5 MAE - MoCo-

### Get Hamming Distances

In [11]:
from scipy.spatial.distance import hamming

In [12]:
for index, key1 in enumerate(keys):
    for key2 in keys[index+1:]:
        dist1 = hamming(dic[key1][0][:, 0], dic[key2][0][:, 0])
        print('Hamming Distance Top-1', key1,'-',key2,'=',dist1)
        dist5 = hamming(dic[key1][0][:, 1], dic[key2][0][:, 1])
        print('Hamming Distance Top-5', key1,'-',key2,'=',dist5)  

Hamming Distance Top-1 supervised - MAE = 0.19822
Hamming Distance Top-5 supervised - MAE = 0.09938
Hamming Distance Top-1 supervised - DINO = 0.10842
Hamming Distance Top-5 supervised - DINO = 0.04244
Hamming Distance Top-1 supervised - MoCo-V3 = 0.12048
Hamming Distance Top-5 supervised - MoCo-V3 = 0.04594
Hamming Distance Top-1 supervised - MAE-FT = 0.08652
Hamming Distance Top-5 supervised - MAE-FT = 0.02982
Hamming Distance Top-1 supervised - DINO-FT = 0.08106
Hamming Distance Top-5 supervised - DINO-FT = 0.02816
Hamming Distance Top-1 supervised - MoCo-V3-FT = 0.08076
Hamming Distance Top-5 supervised - MoCo-V3-FT = 0.02886
Hamming Distance Top-1 MAE - DINO = 0.18008
Hamming Distance Top-5 MAE - DINO = 0.08814
Hamming Distance Top-1 MAE - MoCo-V3 = 0.17274
Hamming Distance Top-5 MAE - MoCo-V3 = 0.08576
Hamming Distance Top-1 MAE - MAE-FT = 0.20722
Hamming Distance Top-5 MAE - MAE-FT = 0.10464
Hamming Distance Top-1 MAE - DINO-FT = 0.20528
Hamming Distance Top-5 MAE - DINO-FT = 0.

## Compare failure modes

In [12]:
from scipy.stats import kendalltau

In [13]:
from tqdm import tqdm

In [14]:
def avg_kt(ranks1, ranks2, top=5, wrong_only=False):

    sum = 0.

    for sample in range(len(ranks1)):
        rank_sim = kendalltau(ranks1[sample, :top], ranks2[sample, :top])
        corr = rank_sim[0]
        if not np.isnan(corr):
            sum += corr

    return sum/50000.

In [15]:
for index, key1 in enumerate(keys):
    for key2 in keys[index+1:]:
        top_5 = avg_kt(dic[key1][1], dic[key2][1])
        print('Average Kendall-Tau Top-5', key1,'-',key2,'=',top_5)
        top_10 = avg_kt(dic[key1][1], dic[key2][1], 10)
        print('Average Kendall-Tau Top-10', key1,'-',key2,'=',top_10)
        top_100 = avg_kt(dic[key1][1], dic[key2][1], 100)
        print('Average Kendall-Tau Top-100', key1,'-',key2,'=',top_100)

Average Kendall-Tau Top-5 supervised - MAE = 0.127656000000001
Average Kendall-Tau Top-10 supervised - MAE = 0.06569333333333374
Average Kendall-Tau Top-100 supervised - MAE = 0.02158213333333343
Average Kendall-Tau Top-5 supervised - DINO = 0.18803600000000112
Average Kendall-Tau Top-10 supervised - DINO = 0.09634133333333386
Average Kendall-Tau Top-100 supervised - DINO = 0.02781517575757575
Average Kendall-Tau Top-5 supervised - MoCo-V3 = 0.1812800000000016
Average Kendall-Tau Top-10 supervised - MoCo-V3 = 0.09365244444444404
Average Kendall-Tau Top-100 supervised - MoCo-V3 = 0.02704848484848447
Average Kendall-Tau Top-5 supervised - MAE-FT = 0.22474399999999792
Average Kendall-Tau Top-10 supervised - MAE-FT = 0.11440888888888727
Average Kendall-Tau Top-100 supervised - MAE-FT = 0.03443250101010133
Average Kendall-Tau Top-5 supervised - DINO-FT = 0.2333199999999968
Average Kendall-Tau Top-10 supervised - DINO-FT = 0.11781688888888542
Average Kendall-Tau Top-100 supervised - DINO-FT 

## Pair-wise Correct Predictions

In [16]:
labels = [x[1] for x in data_loader.dataset.samples]

In [17]:
for index, key1 in enumerate(keys):
    for key2 in keys[index+1:]:
        top_5 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)], dic[key2][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)])
        print('Average Kendall-Tau Top-5', key1,'-',key2,'=',top_5)
        top_10 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)], dic[key2][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)], 10)
        print('Average Kendall-Tau Top-10', key1,'-',key2,'=',top_10)
        top_100 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)], dic[key2][1][(dic[key1][0][:, 0] == 1) & (dic[key2][0][:, 0] == 1)], 100)
        print('Average Kendall-Tau Top-100', key1,'-',key2,'=',top_100)

Average Kendall-Tau Top-5 supervised - MAE = 0.11540000000000031
Average Kendall-Tau Top-10 supervised - MAE = 0.05534088888888877
Average Kendall-Tau Top-100 supervised - MAE = 0.014563434343434304
Average Kendall-Tau Top-5 supervised - DINO = 0.16549600000000184
Average Kendall-Tau Top-10 supervised - DINO = 0.08069511111111273
Average Kendall-Tau Top-100 supervised - DINO = 0.021078658585858688
Average Kendall-Tau Top-5 supervised - MoCo-V3 = 0.15955200000000327
Average Kendall-Tau Top-10 supervised - MoCo-V3 = 0.07757822222222344
Average Kendall-Tau Top-100 supervised - MoCo-V3 = 0.020346359595959505
Average Kendall-Tau Top-5 supervised - MAE-FT = 0.19584400000000132
Average Kendall-Tau Top-10 supervised - MAE-FT = 0.09577555555555588
Average Kendall-Tau Top-100 supervised - MAE-FT = 0.02638634343434363
Average Kendall-Tau Top-5 supervised - DINO-FT = 0.20265999999999934
Average Kendall-Tau Top-10 supervised - DINO-FT = 0.09879466666666627
Average Kendall-Tau Top-100 supervised - D

## Pair-wise Incorrect Predictions

In [18]:
for index, key1 in enumerate(keys):
    for key2 in keys[index+1:]:
        top_5 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)], dic[key2][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)])
        print('Average Kendall-Tau Top-5', key1,'-',key2,'=',top_5)
        top_10 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)], dic[key2][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)], 10)
        print('Average Kendall-Tau Top-10', key1,'-',key2,'=',top_10)
        top_100 = avg_kt(dic[key1][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)], dic[key2][1][(dic[key1][0][:, 0] == 0) & (dic[key2][0][:, 0] == 0)], 100)
        print('Average Kendall-Tau Top-100', key1,'-',key2,'=',top_100)

Average Kendall-Tau Top-5 supervised - MAE = 0.017459999999999965
Average Kendall-Tau Top-10 supervised - MAE = 0.00936799999999999
Average Kendall-Tau Top-100 supervised - MAE = 0.0032268363636363596
Average Kendall-Tau Top-5 supervised - DINO = 0.025155999999999838
Average Kendall-Tau Top-10 supervised - DINO = 0.013484888888888895
Average Kendall-Tau Top-100 supervised - DINO = 0.004130351515151517
Average Kendall-Tau Top-5 supervised - MoCo-V3 = 0.02437999999999998
Average Kendall-Tau Top-10 supervised - MoCo-V3 = 0.013388888888888877
Average Kendall-Tau Top-100 supervised - MoCo-V3 = 0.00404051717171716
Average Kendall-Tau Top-5 supervised - MAE-FT = 0.02903599999999983
Average Kendall-Tau Top-10 supervised - MAE-FT = 0.015290222222222184
Average Kendall-Tau Top-100 supervised - MAE-FT = 0.0049755959595959645
Average Kendall-Tau Top-5 supervised - DINO-FT = 0.031091999999999842
Average Kendall-Tau Top-10 supervised - DINO-FT = 0.016085777777777758
Average Kendall-Tau Top-100 super

## Model-wise correct predictions

### MAE

In [ ]:
top_5 = avg_kt(mae_ranks[mae_results[:, 0] == 1], moco_ranks[mae_results[:, 0] == 1])
top_10 = avg_kt(mae_ranks[mae_results[:, 0] == 1], moco_ranks[mae_results[:, 0] == 1], 10)
top_100 = avg_kt(mae_ranks[mae_results[:, 0] == 1], moco_ranks[mae_results[:, 0] == 1], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[mae_results[:, 0] == 1], dino_ranks[mae_results[:, 0] == 1])
top_10 = avg_kt(mae_ranks[mae_results[:, 0] == 1], dino_ranks[mae_results[:, 0] == 1], 10)
top_100 = avg_kt(mae_ranks[mae_results[:, 0] == 1], dino_ranks[mae_results[:, 0] == 1], 100)

In [ ]:
print(top_5, top_10, top_100)

### MoCo-v3

In [ ]:
top_5 = avg_kt(mae_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1])
top_10 = avg_kt(mae_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1], 10)
top_100 = avg_kt(mae_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(dino_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1])
top_10 = avg_kt(dino_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1], 10)
top_100 = avg_kt(dino_ranks[moco_results[:, 0] == 1], moco_ranks[moco_results[:, 0] == 1], 100)

In [ ]:
print(top_5, top_10, top_100)

## Inorrect Predictions

### MAE

In [ ]:
top_5 = avg_kt(mae_ranks[mae_results[:, 0] == 0], moco_ranks[mae_results[:, 0] == 0])
top_10 = avg_kt(mae_ranks[mae_results[:, 0] == 0], moco_ranks[mae_results[:, 0] == 0], 10)
top_100 = avg_kt(mae_ranks[mae_results[:, 0] == 0], moco_ranks[mae_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[mae_results[:, 0] == 0], dino_ranks[mae_results[:, 0] == 0])
top_10 = avg_kt(mae_ranks[mae_results[:, 0] == 0], dino_ranks[mae_results[:, 0] == 0], 10)
top_100 = avg_kt(mae_ranks[mae_results[:, 0] == 0], dino_ranks[mae_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

### MoCo-v3

In [ ]:
top_5 = avg_kt(mae_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0])
top_10 = avg_kt(mae_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0], 10)
top_100 = avg_kt(mae_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(dino_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0])
top_10 = avg_kt(dino_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0], 10)
top_100 = avg_kt(dino_ranks[moco_results[:, 0] == 0], moco_ranks[moco_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

### DINO

In [ ]:
top_5 = avg_kt(mae_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0])
top_10 = avg_kt(mae_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0], 10)
top_100 = avg_kt(mae_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(moco_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0])
top_10 = avg_kt(moco_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0], 10)
top_100 = avg_kt(moco_ranks[dino_results[:, 0] == 0], dino_ranks[dino_results[:, 0] == 0], 100)

In [ ]:
print(top_5, top_10, top_100)

## Pairwise Correct

In [ ]:
top_5 = avg_kt(moco_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)])
top_10 = avg_kt(moco_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)], 10)
top_100 = avg_kt(moco_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(moco_results[:, 0] == 1) & (dino_results[:, 0] == 1)], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)])
top_10 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)], 10)
top_100 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)], dino_ranks[(mae_results[:, 0] == 1) & (dino_results[:, 0] == 1)], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)], moco_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)])
top_10 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)], moco_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)], 10)
top_100 = avg_kt(mae_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)], moco_ranks[(mae_results[:, 0] == 1) & (moco_results[:, 0] == 1)], 100)

In [ ]:
print(top_5, top_10, top_100)

## Pairwise incorrect

In [ ]:
top_5 = avg_kt(moco_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)])
top_10 = avg_kt(moco_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)], 10)
top_100 = avg_kt(moco_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(moco_results[:, 0] == 0) & (dino_results[:, 0] == 0)], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)])
top_10 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)], 10)
top_100 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)], dino_ranks[(mae_results[:, 0] == 0) & (dino_results[:, 0] == 0)], 100)

In [ ]:
print(top_5, top_10, top_100)

In [ ]:
top_5 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)], moco_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)])
top_10 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)], moco_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)], 10)
top_100 = avg_kt(mae_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)], moco_ranks[(mae_results[:, 0] == 0) & (moco_results[:, 0] == 0)], 100)

In [ ]:
print(top_5, top_10, top_100)

Class-wise accuracy

In [ ]:
mae_classwise = {i:0 for i in range(1000)}
moco_classwise = {i:0 for i in range(1000)}
dino_classwise = {i:0 for i in range(1000)}

mae_classwise_5 = {i:0 for i in range(1000)}
moco_classwise_5 = {i:0 for i in range(1000)}
dino_classwise_5 = {i:0 for i in range(1000)}

In [ ]:
for i, label in enumerate(labels):
    if mae_results[i, 0] == 1:
        mae_classwise[label] += 1
    if moco_results[i, 0] == 1:
        moco_classwise[label] += 1
    if dino_results[i, 0] == 1:
        dino_classwise[label] += 1
    if mae_results[i, 1] == 1:
        mae_classwise_5[label] += 1
    if moco_results[i, 1] == 1:
        moco_classwise_5[label] += 1
    if dino_results[i, 1] == 1:
        dino_classwise_5[label] += 1


In [ ]:
sns.barplot(x=list(mae_classwise.keys()), y=list(mae_classwise.values()))
plt.xticks([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])